In [1]:
import os
import sys
from tqdm import tqdm
import numpy as np
import pandas as pd

from one.api import ONE
from brainbox.io.one import SessionLoader
from iblatlas.regions import BrainRegions

from sklearn import linear_model as sklm
from sklearn.metrics import accuracy_score, balanced_accuracy_score, r2_score
from sklearn.model_selection import KFold, train_test_split
from behavior_models.utils import format_data as format_data_mut
from behavior_models.utils import format_input as format_input_mut

from brainwidemap.bwm_loading import load_good_units, load_all_units, load_trials_and_mask, merge_probes
from brainwidemap.decoding.functions.process_targets import load_behavior
from brainwidemap.decoding.settings_for_BWM_figure.settings_choice import params
from brainwidemap.decoding.settings_for_BWM_figure.settings_choice import RESULTS_DIR

from brainwidemap.decoding.functions.balancedweightings import balanced_weighting
from brainwidemap.decoding.functions.process_inputs import (
    build_predictor_matrix,
    select_ephys_regions,
    preprocess_ephys
)
from brainwidemap.decoding.functions.process_targets import (
    compute_beh_target,
    compute_target_mask,
    transform_data_for_decoding,
    logisticreg_criteria,
    get_target_data_per_trial_wrapper,
    check_bhv_fit_exists,
    optimal_Bayesian
)
from brainwidemap.decoding.functions.utils import save_region_results, get_save_path
from brainwidemap.decoding.functions.nulldistributions import generate_null_distribution_session
from brainwidemap.decoding.functions.decoding import decode_cv

/home/yizi/anaconda3/envs/ibl_bwm/lib/python3.10/site-packages/ibllib/atlas/__init__.py:202: DeprecationWarning: ibllib.atlas is deprecated. Please install iblatlas using "pip install iblatlas" and use this module instead
  warnings.warn('ibllib.atlas is deprecated. Please install iblatlas using "pip install iblatlas" and use '


### load data

In [4]:
params['behfit_path'] = RESULTS_DIR.joinpath('decoding', 'results', 'behavioral')
params['behfit_path'].mkdir(parents=True, exist_ok=True)
params['neuralfit_path'] = RESULTS_DIR.joinpath('decoding', 'results', 'neural')
params['neuralfit_path'].mkdir(parents=True, exist_ok=True)
params['add_to_saving_path'] = (f"_binsize={1000 * params['binsize']}_lags={params['n_bins_lag']}_"
                                f"mergedProbes_{params['merged_probes']}")
imposter_file = RESULTS_DIR.joinpath('decoding', f"imposterSessions_{params['target']}.pqt")
bwm_session_file = RESULTS_DIR.joinpath('decoding', 'bwm_cache_sessions.pqt')

In [5]:
params["align_time"] = "stimOn_times"
# arams["align_time"] = "firstMovement_times"
print(params["align_time"])

stimOn_times


In [6]:
params["time_window"] = (-.1, .0)
# params["time_window"] = (-0.5, 1.)
print(params["time_window"])

(-0.1, 0.0)


In [7]:
params["binsize"] = 0.1
# params["binsize"] = 0.05
print(params["binsize"])
params['n_pseudo'] = 0

0.1


In [8]:
one = ONE(base_url="https://openalyx.internationalbrainlab.org", mode='remote')
bwm_df = pd.read_parquet(bwm_session_file)

In [10]:
pids = one.search_insertions(atlas_acronym=['PO'], query_type='remote')

In [13]:
# list(pids)

In [9]:
bwm_df

,pid,eid,probe_name,session_number,date,subject,lab
0,56f2a378-78d2-4132-b3c8-8c1ba82be598,6713a4a7-faed-4df2-acab-ee4e63326f8d,probe00,1,2020-02-18,NYU-11,angelakilab
1,47be9ae4-290f-46ab-b047-952bc3a1a509,56956777-dca5-468c-87cb-78150432cc57,probe01,1,2020-02-21,NYU-11,angelakilab
2,6be21156-33b0-4f70-9a0f-65b3e3cd6d4a,56956777-dca5-468c-87cb-78150432cc57,probe00,1,2020-02-21,NYU-11,angelakilab
3,1e176f17-d00f-49bb-87ff-26d237b525f1,a8a8af78-16de-4841-ab07-fde4b5281a03,probe00,1,2020-01-22,NYU-12,angelakilab
4,701026df-e170-4ca7-88aa-eb0b95ef6ba1,a8a8af78-16de-4841-ab07-fde4b5281a03,probe01,1,2020-01-22,NYU-12,angelakilab
...,...,...,...,...,...,...,...
542,8bf0f1a4-0d8c-4df3-a99e-f7c81c809652,993c7024-0abc-4028-ad30-d397ad55b084,probe01,1,2020-09-16,CSH_ZAD_029,zadorlab
543,5d570bf6-a4c6-4bf1-a14b-2c878c84ef0e,fece187f-b47f-4870-a1d6-619afe942a7d,probe01,1,2020-09-17,CSH_ZAD_029,zadorlab
544,f7c93877-ec05-4091-a003-e69fae0f2fa8,fece187f-b47f-4870-a1d6-619afe942a7d,probe00,1,2020-09-17,CSH_ZAD_029,zadorlab
545,675952a4-e8b3-4e82-a179-cc970d5a8b01,c7bd79c9-c47e-4ea5-aea3-74dda991b48e,probe01,1,2020-09-19,CSH_ZAD_029,zadorlab


In [7]:
one = ONE(base_url="https://openalyx.internationalbrainlab.org", mode='remote')
bwm_df = pd.read_parquet(bwm_session_file)

In [ ]:
idx = 1

job_repeat = 0 
pseudo_ids = np.arange(job_repeat * params['n_pseudo_per_job'], (job_repeat + 1) * params['n_pseudo_per_job']) + 1
if 1 in pseudo_ids:
    pseudo_ids = np.concatenate((-np.ones(1), pseudo_ids)).astype('int64')
if pseudo_ids[0] > params['n_pseudo']:
    print(f"ended job because this job_repeat ({job_repeat}) does not include any pseudo sessions < {params['n_pseudo']}")
    exit()
if pseudo_ids[-1] > params['n_pseudo']:
    print(f"truncated job because this job_repeat ({job_repeat}) includes more than {params['n_pseudo']} pseudo sessions")
    pseudo_ids = pseudo_ids[pseudo_ids <= params['n_pseudo']]

if params['merged_probes']:
    eid = bwm_df['eid'].unique()[idx]
    tmp_df = bwm_df.set_index(['eid', 'subject']).xs(eid, level='eid')
    subject = tmp_df.index[0]
    pids = tmp_df['pid'].to_list()  # Select all probes of this session
    probe_names = tmp_df['probe_name'].to_list()
    print(f"Running merged probes for session eid: {eid}")
else:
    eid = bwm_df.iloc[idx]['eid']
    subject = bwm_df.iloc[idx]['subject']
    pid = bwm_df.iloc[idx]['pid']
    probe_name = bwm_df.iloc[idx]['probe_name']
    print(f"Running probe pid: {pid}")
    
sess_loader = SessionLoader(one, eid)
sess_loader.load_trials()

trials_df, trials_mask = load_trials_and_mask(
    one=one, eid=eid, sess_loader=sess_loader, min_rt=params['min_rt'], max_rt=params['max_rt'],
    min_trial_len=params['min_len'], max_trial_len=params['max_len'],
    exclude_nochoice=True, exclude_unbiased=params['exclude_unbiased_trials'])
_, trials_mask_without_minrt = load_trials_and_mask(
    one=one, eid=eid, sess_loader=sess_loader, min_rt=None, max_rt=params['max_rt'],
    min_trial_len=params['min_len'], max_trial_len=params['max_len'],
    exclude_nochoice=True, exclude_unbiased=params['exclude_unbiased_trials'])
_, trials_mask_without_maxrt = load_trials_and_mask(
    one=one, eid=eid, sess_loader=sess_loader, min_rt=params['min_rt'], max_rt=None,
    min_trial_len=params['min_len'], max_trial_len=params['max_len'],
    exclude_nochoice=True, exclude_unbiased=params['exclude_unbiased_trials'])
_, trials_mask_withonly_nochoice = load_trials_and_mask(
    one=one, eid=eid, sess_loader=sess_loader, min_rt=None, max_rt=None,
    min_trial_len=None, max_trial_len=None,
    exclude_nochoice=True, exclude_unbiased=False)

params['trials_mask_diagnostics'] = [trials_mask,
                                     trials_mask_without_minrt,
                                     trials_mask_without_maxrt,
                                     trials_mask_withonly_nochoice]

if params['target'] in ['wheel-vel', 'wheel-speed', 'l-whisker-me', 'r-whisker-me']:
    # load target data
    dlc_dict = load_behavior(params['target'], sess_loader)
    # load imposter sessions
    params['imposter_df'] = pd.read_parquet(imposter_file) if params['n_pseudo'] > 0 else None
else:
    dlc_dict = None
    params['imposter_df'] = None

if params['merged_probes']:
    clusters_list = []
    spikes_list = []
    for pid, probe_name in zip(pids, probe_names):
        tmp_spikes, tmp_clusters = load_good_units(one, pid, eid=eid, pname=probe_name)
        # tmp_spikes, tmp_clusters = load_all_units(one, pid, eid=eid, pname=probe_name)
        tmp_clusters['pid'] = pid
        spikes_list.append(tmp_spikes)
        clusters_list.append(tmp_clusters)
    spikes, clusters = merge_probes(spikes_list, clusters_list)
else:
    spikes, clusters = load_good_units(one, pid, eid=eid, pname=probe_name)
    # spikes, clusters = load_all_units(one, pid, eid=eid, pname=probe_name)

neural_dict = {
    'spk_times': spikes['times'],
    'spk_clu': spikes['clusters'],
    'clu_regions': clusters['acronym'],
    'clu_qc': {k: np.asarray(v) for k, v in clusters.to_dict('list').items()},
    'clu_df': clusters
}

metadata = {
    'subject': subject,
    'eid': eid,
    'probe_name': probe_name
}

In [9]:
kwargs = params
# pseudo_id = -1
# pseudo_ids = -np.ones(1).astype('int64')
kwargs['n_runs'] = 1
kwargs['n_bins_lag'] = 0

In [10]:
print(f'Working on eid: {metadata["eid"]}')
filenames = []  # this will contain paths to saved decoding results for this eid

if kwargs['use_imposter_session'] and not kwargs['stitching_for_imposter_session']:
    trials_df = trials_df[:int(kwargs['max_number_trials_when_no_stitching_for_imposter_session'])]

if 0 in pseudo_ids:
    raise ValueError(
        'pseudo id can be -1 (actual session) or strictly greater than 0 (pseudo session)')

if not np.all(np.sort(pseudo_ids) == pseudo_ids):
    raise ValueError('pseudo_ids must be sorted')

if kwargs['model'] == optimal_Bayesian and np.any(trials_df.probabilityLeft.values[:90] != 0.5):
    raise ValueError(
        'The optimal Bayesian model assumes 90 unbiased trials at the beginning of the '
        'session, which is not the case here.')
    
# check if is trained
eids_train = (
    [metadata['eid']] if 'eids_train' not in metadata.keys() else metadata['eids_train'])
if 'eids_train' not in metadata.keys():
    metadata['eids_train'] = eids_train
elif metadata['eids_train'] != eids_train:
    raise ValueError(
        'eids_train are not supported yet. If you do not understand this error, '
        'just take out the eids_train key in the metadata to solve it')
    
if isinstance(kwargs['model'], str):
    import pickle
    from braindelphi.params import INTER_INDIVIDUAL_PATH
    inter_individual = pickle.load(open(INTER_INDIVIDUAL_PATH.joinpath(kwargs['model']), 'rb'))
    if metadata['eid'] not in inter_individual.keys():
        logging.exception('no inter individual model found')
        print(filenames)
    inter_indiv_model_specifications = inter_individual[metadata['eid']]
    print('winning interindividual model is %s' % inter_indiv_model_specifications['model_name'])
    if inter_indiv_model_specifications['model_name'] not in kwargs['modeldispatcher'].values():
        logging.exception('winning inter individual model is LeftKernel or RightKernel')
        print(filenames)
    kwargs['model'] = {v: k for k, v in kwargs['modeldispatcher'].items()}[inter_indiv_model_specifications['model_name']]
    kwargs['model_parameters'] = inter_indiv_model_specifications['model_parameters']
else:
    kwargs['model_parameters'] = None
    # train model if not trained already
    if kwargs['model'] != optimal_Bayesian and kwargs['model'] is not None:
        side, stim, act, _ = format_data_mut(trials_df)
        stimuli, actions, stim_side = format_input_mut([stim], [act], [side])
        behmodel = kwargs['model'](
            kwargs['behfit_path'], np.array(metadata['eids_train']), metadata['subject'],
            actions, stimuli, trials_df, stim_side, single_zeta=True)
        istrained, _ = check_bhv_fit_exists(
            metadata['subject'], kwargs['model'], metadata['eids_train'],
            kwargs['behfit_path'], modeldispatcher=kwargs['modeldispatcher'], single_zeta=True)
        if not istrained:
            behmodel.load_or_train(remove_old=False)

if kwargs['balanced_weight'] and kwargs['balanced_continuous_target']:
    raise NotImplementedError("see tag `decoding_biasCWnull` for a previous implementation.")
else:
    target_distribution = None
    
# get target values
if kwargs['target'] in ['pLeft', 'signcont', 'strengthcont', 'choice', 'feedback']:
    target_vals_list, target_vals_to_mask = compute_beh_target(
        trials_df, metadata, return_raw=True, **kwargs)
    target_mask = compute_target_mask(
        target_vals_to_mask, kwargs['exclude_trials_within_values'])

else:
    if dlc_dict is None or dlc_dict['times'] is None or dlc_dict['values'] is None:
        raise ValueError('dlc_dict does not contain any data')
    _, target_vals_list, target_mask = get_target_data_per_trial_wrapper(
        target_times=dlc_dict['times'],
        target_vals=dlc_dict['values'],
        trials_df=trials_df,
        align_event=kwargs['align_time'],
        align_interval=kwargs['time_window'],
        binsize=kwargs['binsize'])
    
mask = trials_mask & target_mask

if sum(mask) <= kwargs['min_behav_trials']:
    msg = 'session contains %i trials, below the threshold of %i' % (
        sum(mask), kwargs['min_behav_trials'])
    logging.exception(msg)
    print(filenames)

Working on eid: 56956777-dca5-468c-87cb-78150432cc57


In [11]:
# select brain regions from beryl atlas to loop over
brainreg = BrainRegions()
beryl_reg = brainreg.acronym2acronym(neural_dict['clu_regions'], mapping='Beryl')
regions = (
    [[k] for k in np.unique(beryl_reg)] if kwargs['single_region'] else [np.unique(beryl_reg)])

In [12]:
region_results = {}
# for region in tqdm(regions, desc='Region: ', leave=False):
for region in tqdm(["GPe"], desc='Region: ', leave=False):

    # pull spikes from this region out of the neural data
    reg_clu_ids = select_ephys_regions(neural_dict, beryl_reg, region, **kwargs)

    # skip region if there are not enough units
    n_units = len(reg_clu_ids)
    if n_units < kwargs['min_units']:
        continue

    # bin spikes from this region for each trial
    msub_binned, cl_inds_used = preprocess_ephys(reg_clu_ids, neural_dict, trials_df, **kwargs)
    cl_uuids_used = list(neural_dict['clu_df'].iloc[cl_inds_used]['uuids'])

    # make design matrix
    bins_per_trial = msub_binned[0].shape[0]
    Xs = (
        msub_binned if bins_per_trial == 1
        else [build_predictor_matrix(s, kwargs['n_bins_lag']) for s in msub_binned]
    )
    
    for pseudo_id in pseudo_ids:
        fit_results = []

        # save out decoding results
        save_path = get_save_path(
            pseudo_id, metadata['subject'], metadata['eid'], 'ephys',
            probe=metadata['probe_name'],
            region=str(np.squeeze(region)) if kwargs['single_region'] else 'allRegions',
            output_path=kwargs['neuralfit_path'],
            time_window=kwargs['time_window'],
            date=kwargs['date'],
            target=kwargs['target'],
            add_to_saving_path=kwargs['add_to_saving_path']
        )
        if os.path.exists(save_path):
            print(
                f'results for region {region}, pseudo_id {pseudo_id} already exist at '
                f'{save_path}')
            filenames.append(save_path)
            continue

        # get data matrix and target, resampling when there are <3 incorrect trials

        # create pseudo/imposter session when necessary, corresponding mask, and data matrix
        if pseudo_id > 0:
            ys_wmask = None
            Xs_wmask = None
            sample_pseudo_count = 0
            while (kwargs['estimator']==sklm.LogisticRegression and (not logisticreg_criteria(ys_wmask))) or (ys_wmask is None):
                assert sample_pseudo_count < 100 # must be a reasonable number of sample or else something is wrong
                sample_pseudo_count += 1
                if bins_per_trial == 1:
                    controlsess_df = generate_null_distribution_session(
                        trials_df, metadata, **kwargs)
                    controltarget_vals_list, controltarget_vals_to_mask = compute_beh_target(
                        controlsess_df, metadata, return_raw=True, **kwargs)
                    controltarget_mask = compute_target_mask(
                        controltarget_vals_to_mask, kwargs['exclude_trials_within_values'])
                    control_mask = trials_mask & controltarget_mask
                else:
                    imposter_df = kwargs['imposter_df'].copy()
                    # remove current eid from imposter sessions
                    df_clean = imposter_df[imposter_df.eid != metadata['eid']].reset_index()
                    # randomly select imposter trial to start sequence
                    n_trials = trials_df.index.size
                    total_imposter_trials = df_clean.shape[0]
                    idx_beg = np.random.choice(total_imposter_trials - n_trials)
                    controlsess_df = df_clean.iloc[idx_beg:idx_beg + n_trials]
                    # grab target values from this dataframe
                    controltarget_vals_list = list(controlsess_df[kwargs['target']].to_numpy())
                    control_mask = mask

                save_predictions = kwargs.get('save_predictions_pseudo', kwargs['save_predictions'])

                # session for null dist
                ys_wmask = [controltarget_vals_list[m] for m in np.squeeze(np.where(control_mask))]
                Xs_wmask = [Xs[m] for m in np.squeeze(np.where(control_mask))]

            if sample_pseudo_count > 1:
                print(f'sampled pseudo sessions {sample_pseudo_count} times to ensure valid target')

        else:
            control_mask = mask
            save_predictions = kwargs['save_predictions']

            # original session
            ys_wmask = [target_vals_list[m] for m in np.squeeze(np.where(mask))]
            Xs_wmask = [Xs[m] for m in np.squeeze(np.where(mask))]

            if kwargs['estimator'] == sklm.LogisticRegression and (not logisticreg_criteria(ys_wmask)):
                print(f'target failed logistic regression criteria for region {region} and pseudo_id {pseudo_id}')
                continue
    
        for i_run in range(kwargs['n_runs']):

            rng_seed = i_run

            fit_result = decode_cv(
                ys=ys_wmask,
                Xs=Xs_wmask,
                estimator=kwargs['estimator'],
                use_openturns=kwargs['use_openturns'],
                target_distribution=target_distribution,
                balanced_continuous_target=kwargs['balanced_continuous_target'],
                estimator_kwargs=kwargs['estimator_kwargs'],
                hyperparam_grid=kwargs['hyperparam_grid'],
                save_binned=kwargs['save_binned'] if pseudo_id == -1 else False,
                save_predictions=save_predictions,
                shuffle=kwargs['shuffle'],
                balanced_weight=kwargs['balanced_weight'],
                rng_seed=rng_seed,
            )
            fit_result['mask'] = mask
            fit_result['mask_trials_and_targets'] = [trials_mask, target_mask]
            fit_result['mask_diagnostics'] = kwargs['trials_mask_diagnostics']
            fit_result['df'] = trials_df if pseudo_id == -1 else controlsess_df
            fit_result['pseudo_id'] = pseudo_id
            fit_result['run_id'] = i_run
            fit_result['cluster_uuids'] = cl_uuids_used
            fit_results.append(fit_result)
            
        filename = save_region_results(
                fit_result=fit_results,
                pseudo_id=pseudo_id,
                subject=metadata['subject'],
                eid=metadata['eid'],
                probe=metadata['probe_name'],
                region=region,
                n_units=n_units,
                save_path=save_path
        )
        
        filenames.append(filename)
        
print(f'Finished eid: {metadata["eid"]}')

Region:   0%|                                             | 0/1 [00:00<?, ?it/s]

results for region GPe, pseudo_id -1 already exist at /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/neural/ephys/NYU-11/56956777-dca5-468c-87cb-78150432cc57/probe00/01-04-2023_GPe_target_choice_timeWindow_-0_1_0_0_pseudo_id_-1__binsize=50.0_lags=None_mergedProbes_True.pkl
2023-11-08 12:37:43.986 INFO     [base_models.py:  289]   results found and loaded from /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/behavioral/NYU-11/model_actKernel_single_zeta/train_56956777.pkl


/home/yizi/decode-brain-wide-map/brainwidemap/decoding/functions/nulldistributions.py:61: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  act_sim = np.array(act_sim.squeeze().T, dtype=np.int64)


2023-11-08 12:37:44.580 INFO     [base_models.py:  289]   results found and loaded from /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/behavioral/NYU-11/model_actKernel_single_zeta/train_56956777.pkl
2023-11-08 12:37:45.141 INFO     [base_models.py:  289]   results found and loaded from /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/behavioral/NYU-11/model_actKernel_single_zeta/train_56956777.pkl
2023-11-08 12:37:45.702 INFO     [base_models.py:  289]   results found and loaded from /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/behavioral/NYU-11/model_actKernel_single_zeta/train_56956777.pkl
2023-11-08 12:37:46.255 INFO     [base_models.py:  289]   results found and loaded from /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/behavioral/NYU-11/model_actKernel_single_zeta/train_56956777.pkl
2023-11-08 12:37:46.813 INFO     [base_models.py:  289]   results found and loaded from /mnt/3TB/yizi/decode-paper-brain-wide-map/decoding/results/behav

Finished eid: 56956777-dca5-468c-87cb-78150432cc57


In [33]:
res = []
for filename in filenames:
    with open(filename, 'rb') as f:
        res.append(pickle.load(f))
        
true_score = res[0]["fit"][0]["scores_test_full"]
null_median = np.median([null["fit"][0]["scores_test_full"] for null in res[1:]])
adj_acc = true_score - null_median
print(true_score, adj_acc)

0.6627795962902346 0.17604144197544624
